# Início

## Configuração inicial

In [12]:
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [13]:
seed = 21
np.random.seed(seed)

## Obtenção dos dados

In [17]:
import pandas as pd

url = "https://raw.githubusercontent.com/amandatz/employee-attrition-analysis/main/data/data.csv"
employees = pd.read_csv(url)


## Descrição

O conjunto possui 30 colunas, sendo elas:

1. **Age**: Idade do empregado;
2. **Attrition**: Se o empregado deixou a empresa no último ano ou não;
3. **BusinessTravel**: Frequência com que os empregados viajaram para fins de negócios no último ano;
4. **Department**: Departamento em que o empregado trabalha;
5. **DistanceFromHome**: Distância de casa em quilômetros;
6. **Education**: Nível de escolaridade do empregado;
  1. Abaixo do Ensino Superior
  2. Ensino Superior
  3. Bacharelado
  4. Mestrado
  5. Doutorado
7. **EducationField**: Área de formação do empregado;
8. **EmployeeCount**: Contagem de empregados;
9. **EmployeeNumber**: Número ou ID do empregado;
10. **EnvironmentSatisfaction**: Nível de satisfação com o ambiente de trabalho;
  1. Baixo
  2. Médio
  3. Alto
  4. Muito Alto
11. **Gender**: Gênero do empregado;
12. **JobInvolvement**: Nível de envolvimento com o trabalho;
  1. Baixo
  2. Médio
  3. Alto
  4. Muito Alto
13. **JobLevel**: Nível de cargo na empresa, em uma escala de 1 a 5;
14. **JobRole**: Cargo ocupado pelo empregado na empresa;
15. **JobSatisfaction**: Nível de satisfação com o trabalho;
  1. Baixo
  2. Médio
  3. Alto
  4. Muito Alto
16. **MaritalStatus**: Estado civil do empregado;
17. **MonthlyIncome**: Renda mensal em rúpias (Moeda Indiana);
18. **NumCompaniesWorked**: Número total de empresas nas quais o empregado trabalhou;
19. **Over18**: Indica se o empregado tem mais de 18 anos ou não;
20. **PercentSalaryHike**: Percentual de aumento salarial no último ano;
21. **PerformanceRating**: Avaliação de desempenho do último ano;
  1. Baixo
  2. Bom
  3. Excelente
  4. Excepcional
22. **RelationshipSatisfaction**: Nível de satisfação com as relações de trabalho;
  1. Baixo
  2. Médio
  3. Alto
  4. Muito Alto
23. **StandardHours**: Número de horas padrão de trabalho para o empregado;
24. **StockOptionLevel**: Nível de opções de ações do empregado;
25. **TotalWorkingYears**: Número total de anos que o empregado trabalhou;
26. **TrainingTimesLastYear**: Número de vezes que o empregado participou de treinamentos no último ano;
27. **WorkLifeBalance**: Nível de equilíbrio entre vida pessoal e profissional;
  1. Ruim
  2. Bom
  3. Melhor
  4. Ótimo
28. **YearsAtCompany**: Número total de anos que o empregado trabalhou na empresa;
29. **YearsSinceLastPromotion**: Número de anos desde a última promoção;
30. **YearsWithCurrManager**: Número de anos com o atual gerente.









In [18]:
employees.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


## Separação dos dados

## Visualização do conjunto de dados

## Preparação dos dados

Existem dados faltantes. Veja que `NumCompaniesWorked` e `TotalWorkingYears` possuem menos que 4410 registros.


In [15]:
print(df.count())

Age                        4410
Attrition                  4410
BusinessTravel             4410
Department                 4410
DistanceFromHome           4410
Education                  4410
EducationField             4410
EmployeeCount              4410
EmployeeID                 4410
Gender                     4410
JobLevel                   4410
JobRole                    4410
MaritalStatus              4410
MonthlyIncome              4410
NumCompaniesWorked         4391
Over18                     4410
PercentSalaryHike          4410
StandardHours              4410
StockOptionLevel           4410
TotalWorkingYears          4401
TrainingTimesLastYear      4410
YearsAtCompany             4410
YearsSinceLastPromotion    4410
YearsWithCurrManager       4410
dtype: int64


## Comparação entre modelos


## Ajustes e seleção do modelo

## Modelo e teste final